In [1]:
import pickle 
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.utils import np_utils
from keras.initializers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.applications.mobilenet import MobileNet

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from art.attacks import FastGradientMethod
from art.attacks import ProjectedGradientDescent
from art.classifiers import KerasClassifier

/data/aminul/anaconda3/envs/tf_1.4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/aminul/anaconda3/envs/tf_1.4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/aminul/anaconda3/envs/tf_1.4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/aminul/anaconda3/envs/tf_1.4/lib/python3

In [2]:
with open('skin-cancer-dataset', 'rb') as f:
    x_train,x_test,x_val = pickle.load(f)

In [3]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(8111, 100, 75, 3)
(1002, 100, 75, 3)
(902, 100, 75, 3)


In [4]:
with open('skin-cancer-dataset-label', 'rb') as f:
    y_train,y_test,y_val = pickle.load(f)

In [5]:
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(8111, 7)
(1002, 7)
(902, 7)


In [6]:
import cv2
dim=(224,224)
x_new_train=[]
x_new_test=[]
x_new_val=[]

In [7]:
for i in range(8111):
    resized = cv2.resize(x_train[i], dim, interpolation = cv2.INTER_AREA)
    x_new_train.append(resized)
    
for i in range(1002):
    resized = cv2.resize(x_test[i], dim, interpolation = cv2.INTER_AREA)
    x_new_test.append(resized)
    
for i in range(902):
    resized = cv2.resize(x_val[i], dim, interpolation = cv2.INTER_AREA)
    x_new_val.append(resized)

In [8]:
x_new_train = np.array(x_new_train)
x_new_test = np.array(x_new_test)
x_new_val = np.array(x_new_val)

In [9]:
x_new_train = np.concatenate((x_new_train,x_new_val),axis=0)
y_train = np.concatenate((y_train,y_val),axis=0)

In [10]:
print(x_new_train.shape)
print(x_new_test.shape)

(9013, 224, 224, 3)
(1002, 224, 224, 3)


In [12]:
mob1 = MobileNet(weights=None,input_shape=(224,224,3), classes=7)

In [13]:
opt = SGD(lr=0.1,decay=0.002)
mob1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
lrr = ReduceLROnPlateau(monitor='val_acc', 
                        patience=3, 
                        verbose=1, 
                        factor=0.5, 
                        min_lr=0.0001)

In [14]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images
datagen.fit(x_new_train)

In [15]:
cb = EarlyStopping(patience=5,verbose =1)
batch_size=8
history = mob1.fit_generator(
    datagen.flow(x_new_train,y_train, batch_size=8),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=20,
    validation_data=(x_new_val,y_val),
    validation_steps=x_new_val.shape[0] // batch_size
    ,callbacks=[lrr]
)

Epoch 1/20
1013/1013 [==============================] - 102s 100ms/step - loss: 1.0636 - acc: 0.6579 - val_loss: 0.9038 - val_acc: 0.6718
Epoch 2/20
1013/1013 [==============================] - 94s 93ms/step - loss: 0.8770 - acc: 0.6840 - val_loss: 0.8307 - val_acc: 0.6840
Epoch 3/20
1013/1013 [==============================] - 96s 94ms/step - loss: 0.8149 - acc: 0.7012 - val_loss: 0.7845 - val_acc: 0.6874
Epoch 4/20
1013/1013 [==============================] - 96s 94ms/step - loss: 0.8004 - acc: 0.7089 - val_loss: 1.0118 - val_acc: 0.6685
Epoch 5/20
1013/1013 [==============================] - 101s 100ms/step - loss: 0.7694 - acc: 0.7175 - val_loss: 0.7392 - val_acc: 0.6984
Epoch 6/20
1013/1013 [==============================] - 100s 99ms/step - loss: 0.7469 - acc: 0.7294 - val_loss: 0.7812 - val_acc: 0.7051
Epoch 7/20
1013/1013 [==============================] - 98s 97ms/step - loss: 0.7270 - acc: 0.7295 - val_loss: 0.7126 - val_acc: 0.7395
Epoch 8/20
1013/1013 [=====================

In [16]:
print(mob1.evaluate(x_new_test,y_test))

1002/1002 [==============================] - 4s 4ms/step
[0.6448952800737408, 0.772455089701388]


## Model Robustness Across MobileNet PGD Images ##

In [19]:
with open('new_adversarial/adversarial_dense_train1', 'rb') as f:
    k = pickle.load(f)
with open('new_adversarial/adversarial_dense_train2', 'rb') as f:
    l = pickle.load(f)
with open('new_adversarial/adversarial_dense_train3', 'rb') as f:
    m = pickle.load(f)
with open('new_adversarial/adversarial_dense_train4', 'rb') as f:
    n = pickle.load(f)

with open('new_adversarial/adversarial_dense_test', 'rb') as f1:
    x_adv_test = pickle.load(f1)

with open('new_adversarial/adversarial_dense_trainY', 'rb') as f:
    y_train_mob = pickle.load(f)
with open('new_adversarial/adversarial_dense_testY', 'rb') as f:
    y_test_mob = pickle.load(f)
    
k = np.array(k)
l = np.array(l)
m = np.array(m)
n = np.array(n)
x_adv_test = np.array(x_adv_test)
y_train_mob = np.array(y_train_mob)
y_test_mob = np.array(y_test_mob )

In [20]:
k = k.reshape(2254,224,224,3)
l = l.reshape(2253,224,224,3)
m = m.reshape(2253,224,224,3)
n = n.reshape(2253,224,224,3)

x_adv_test_mob = x_adv_test.reshape(1002,224,224,3)

y_adv_train_mob = y_train_mob.reshape(9013,7)
y_adv_test_mob = y_test_mob.reshape(1002,7)

In [21]:
print(k.shape)
print(l.shape)
print(m.shape)
print(n.shape)
print(x_adv_test_mob.shape)
print(y_adv_train_mob.shape)
print(y_adv_test_mob.shape)

(2254, 224, 224, 3)
(2253, 224, 224, 3)
(2253, 224, 224, 3)
(2253, 224, 224, 3)
(1002, 224, 224, 3)
(9013, 7)
(1002, 7)


In [22]:
x_adv_train_mob = np.concatenate((k,l,m,n),axis=0)

In [23]:
print("MobileNet Testing Score Before Adversarial Training")
print(mob1.evaluate(x_adv_test_mob,y_adv_test_mob))

MobileNet Testing Score Before Adversarial Training
1002/1002 [==============================] - 2s 2ms/step
[11.068111703305425, 0.028942115783334492]


## Model Robustness Against VGGNet PGD Adversarial Images ##

In [25]:
with open('new_adversarial/adversarial_vgg_train1', 'rb') as f:
    k = pickle.load(f)
with open('new_adversarial/adversarial_vgg_train2', 'rb') as f:
    l = pickle.load(f)
with open('new_adversarial/adversarial_vgg_train3', 'rb') as f:
    m = pickle.load(f)
with open('new_adversarial/adversarial_vgg_train4', 'rb') as f:
    n = pickle.load(f)

with open('new_adversarial/adversarial_vgg_test', 'rb') as f1:
    x_adv_test = pickle.load(f1)

with open('new_adversarial/adversarial_vgg_trainY', 'rb') as f:
    y_train_vgg = pickle.load(f)
with open('new_adversarial/adversarial_vgg_testY', 'rb') as f:
    y_test_vgg = pickle.load(f)
    
k = np.array(k)
l = np.array(l)
m = np.array(m)
n = np.array(n)
x_adv_test = np.array(x_adv_test)
y_train_vgg = np.array(y_train_vgg)
y_test_vgg = np.array(y_test_vgg )

In [26]:
k = k.reshape(2254,224,224,3)
l = l.reshape(2253,224,224,3)
m = m.reshape(2253,224,224,3)
n = n.reshape(2253,224,224,3)

x_adv_test_vgg = x_adv_test.reshape(1002,224,224,3)

y_adv_train_vgg = y_train_vgg.reshape(9013,7)
y_adv_test_vgg = y_test_vgg.reshape(1002,7)

In [27]:
print(k.shape)
print(l.shape)
print(m.shape)
print(n.shape)
print(x_adv_test.shape)
print(y_train.shape)
print(y_test.shape)

(2254, 224, 224, 3)
(2253, 224, 224, 3)
(2253, 224, 224, 3)
(2253, 224, 224, 3)
(1, 1002, 224, 224, 3)
(9013, 7)
(1002, 7)


In [28]:
x_adv_train_vgg = np.concatenate((k,l,m,n),axis=0)

In [29]:
print("MobileNet Testing Score Before Adversarial Training")
print(mob1.evaluate(x_adv_test_vgg,y_adv_test_vgg))

MobileNet Testing Score Before Adversarial Training
1002/1002 [==============================] - 2s 2ms/step
[6.469197483595736, 0.06087824352784547]


## Adversarial Training ##

In [31]:
x_adv_train_whole= np.concatenate((x_new_train,x_adv_train_mob),axis=0)
y_adv_train_whole = np.concatenate((y_train,y_adv_train_mob),axis=0)

In [32]:
print(x_adv_train_whole.shape)
print(y_adv_train_whole.shape)

(18026, 224, 224, 3)
(18026, 7)


In [33]:
mob2 = MobileNet(weights=None,input_shape=(224,224,3), classes=7)

In [34]:
opt = SGD(lr=0.1,decay=0.002)
mob2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
lrr = ReduceLROnPlateau(monitor='val_acc', 
                        patience=3, 
                        verbose=1, 
                        factor=0.5, 
                        min_lr=0.0001)

In [35]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images
datagen.fit(x_adv_train_whole)

In [36]:
cb = EarlyStopping(patience=5,verbose =1)
batch_size=8
history = mob2.fit_generator(
    datagen.flow(x_adv_train_whole,y_adv_train_whole, batch_size=8),
    steps_per_epoch=x_adv_train_whole.shape[0] // batch_size,
    epochs=20,
    validation_data=(x_new_val,y_val),
    validation_steps=x_new_val.shape[0] // batch_size
    ,callbacks=[lrr]
)

Epoch 1/20
2253/2253 [==============================] - 220s 97ms/step - loss: 1.0417 - acc: 0.6634 - val_loss: 0.8613 - val_acc: 0.6752
Epoch 2/20
2253/2253 [==============================] - 214s 95ms/step - loss: 0.8897 - acc: 0.6889 - val_loss: 0.7720 - val_acc: 0.6984
Epoch 3/20
2253/2253 [==============================] - 217s 96ms/step - loss: 0.8565 - acc: 0.6951 - val_loss: 0.7727 - val_acc: 0.7051
Epoch 4/20
2253/2253 [==============================] - 211s 94ms/step - loss: 0.8357 - acc: 0.7046 - val_loss: 0.7555 - val_acc: 0.7051
Epoch 5/20
2253/2253 [==============================] - 215s 95ms/step - loss: 0.8311 - acc: 0.7077 - val_loss: 0.7563 - val_acc: 0.7118
Epoch 6/20
2253/2253 [==============================] - 216s 96ms/step - loss: 0.8148 - acc: 0.7111 - val_loss: 0.7293 - val_acc: 0.7217
Epoch 7/20
2253/2253 [==============================] - 217s 96ms/step - loss: 0.8076 - acc: 0.7113 - val_loss: 0.7365 - val_acc: 0.7084
Epoch 8/20
2253/2253 [===================

In [37]:
print(mob2.evaluate(x_new_test,y_test))

1002/1002 [==============================] - 3s 3ms/step
[0.648247743438104, 0.7614770461461263]


## Model Robustness Across After Training MobileNet PGD Images ##

In [38]:
print("MobileNet Testing Score Before Adversarial Training")
print(mob2.evaluate(x_adv_test_mob,y_adv_test_mob))

MobileNet Testing Score Before Adversarial Training
1002/1002 [==============================] - 2s 2ms/step
[0.6760566689059168, 0.7594810378051804]


## Model Robustness Across After Training VGG PGD Images ##

In [40]:
print("MobileNet Testing Score Before Adversarial Training")
print(mob2.evaluate(x_adv_test_vgg,y_adv_test_vgg))

MobileNet Testing Score Before Adversarial Training
1002/1002 [==============================] - 2s 2ms/step
[1.3095231041936817, 0.6736526944918071]


## FGSM Results ##

In [43]:
from art.attacks import FastGradientMethod

In [44]:
min_pixel_value = 0.0
max_pixel_value = 1.0
classifier_1 = KerasClassifier(model= mob1, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)

In [45]:
eps=0.3
attack_FGSM = FastGradientMethod(classifier=classifier_1)
x_adv_test_FGSM = attack_FGSM.generate(x=x_new_test)

In [47]:
print(mob1.evaluate(x_adv_test_FGSM,y_test))

1002/1002 [==============================] - 2s 2ms/step
[12.184666171045361, 0.024950099815270618]
